In [96]:
# All libraries:

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import datetime as dt
import requests
import mysql
import mysql.connector

import webbrowser
import time

import smtplib
import ssl
from email.message import EmailMessage

In [97]:
# Connect to database:


conn = mysql.connector.connect(
    host= "127.0.0.1",
    user = "watchsearch", 
    password = "watchsearch",
    database = "watchsearch"
)

cur = conn.cursor()



In [ ]:
alerter_query = """SELECT AlerterID FROM alerter"""

cur.execute(alerter_query)
ids = cur.fetchall()
print(ids)

#while True:
for id in ids:
    alertnr = id[0]

In [131]:
# get URL for scraping
alertnr = 2

sql_url_query = """SELECT watchurl, user_fk FROM alerter WHERE AlerterID = %s """
cur.execute(sql_url_query, (alertnr, ))
r = cur.fetchall()
conn.commit()
for x in r:
    URL = x[0]
    user = x[1]
print(URL)
print(user)

https://www.chrono24.at/search/index.htm?caseMaterials=7&countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=221&maxAgeInDays=0&models=2&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
2


In [132]:
# scrape URL and save (only) new entries to database


html = requests.get(URL)
bs = BeautifulSoup(html.content, 'html.parser')
# gefundene url zwischenspeichern
newEntries = []
for entry in bs.find_all('a', class_ = 'article-item block-item rcard'):
    link = entry['href']
    cur.execute("SELECT * FROM url WHERE ErgebnisURL = %s", (link, ))
    if cur.fetchone() is None:
        cur.execute("""INSERT INTO url ( ErgebnisURL, Alerter_fk, user_fk) VALUES (%s, %s, %s)""", (link, alertnr, user))
        conn.commit()
        newEntries.append(link)
        


print(len(newEntries))
print(newEntries)

46
['/rolex/rolex-daytona-platin-chrono-baguette-dial---like-new--id20714331.htm', '/rolex/rolex-daytona-platin-baguette--id26921817.htm', '/rolex/daytona-ref-116506-950-platinum-diamond-dial-102022-eu-unworn--id26690174.htm', '/rolex/rolex-daytona-116506-baguette--neu--2022--full-set--id23973339.htm', '/rolex/daytona-platin-arabic-dial-2020---116506-verklebt--id26240103.htm', '/rolex/rolex-daytona-like-new-aus-2020--id23789322.htm', '/rolex/rolex-daytona-platin-uhr-116506-papiere-box-neu-ungetragen-2022--id25206335.htm', '/rolex/rolex-daytona-platinum---ice-blue-dial--id26529823.htm', '/rolex/rolex-daytona--platin--ice-blue-baguette-dial--box--papers--2021--id25736093.htm', '/rolex/cosmograph-daytona---full-set----ref-116506--id25574393.htm', '/rolex/daytona---ref-116506---box-amp-de-papiere---950er-platin---baguette-indizes---automatik---jahr-2022--id23885824.htm', '/rolex/daytona-cosmonaute-ref-116506-950-platinum-box-papers-bj-2019-lc100-germany-excellent--id25962024.htm', '/rolex/

In [133]:

sql_url_query = """SELECT ErgebnisURL FROM url WHERE Alerter_fk = 2 """
cur.execute(sql_url_query)
r = cur.fetchall()
conn.commit()
print(len(r))
print(r)

69
[('/rolex/116519-daytona-beach-blue-nos-full-set-eu--sofort--id8999015.htm',), ('/rolex/rolex-rolex-daytona-116519-black-dial-serial-y--id26216516.htm',), ('/rolex/daytona-cosmograph-ref-16519-18k-white-gold-white-dial-folding-clasp-box-bj-1998--id26328041.htm',), ('/rolex/rolex-daytona-paul-newman-dial-yellow-gold-rare-model-with-factory-crocodile-strap-y-serial--id25399084.htm',), ('/rolex/rolex-cosmograph-daytona-lc100-cal-4030--id26482958.htm',), ('/rolex/rolex-cosmograph-daytona-116515ln-rosegold--id26735035.htm',), ('/rolex/rolex-daytona-cosmograph-18k-gelbgold-unpolished-boxpap-2012-lc170-ref-116518--id25970729.htm',), ('/rolex/daytona-elfenbein-18k-rosegold-oyster-perpetual-ref-116515ln-bp-2016--id25534795.htm',), ('/rolex/rolex-daytona-beach-green-chronograph-116519--id25459541.htm',), ('/rolex/rolex-daytona-cosmograph-meteorite-gold-leather-bracelet--id24358220.htm',), ('/rolex/daytona-ref16528--id26248653.htm',), ('/rolex/daytona-cosmograph-ref-116518-18k-white-gold-rubel

In [100]:
# wenn neue url gefunden: email wird geschickt 
# Abfragen der e-mail aus Datenbank 

mail_query = "SELECT email FROM user WHERE userID = %s"
cur.execute(mail_query, (user, ))
c = cur.fetchone()
receiver = str(c[0])
conn.commit

print(receiver)


wi19b075@technikum-wien.at


In [101]:

# email Verschicken

# Define email sender and receiver
email_sender = 'searchforchrono@gmail.com'
email_password = 'xqmdjtzjznhviung'
email_receiver = receiver

# Set the subject and body of the email
subject = 'We found something new for you!'
body = 'Go to these links: \n' 
for url in newEntries: 
    body += "https://www.chrono24.at" + url + '\n'
    
em = EmailMessage()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = subject
em.set_content(body)

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receiver, em.as_string())

In [104]:

    getURL(alertnr)
    scrapUrl()
    getReceiver()
    sendAlert()
    print(URL)



[(1,), (2,), (3,), (4,), (5,)]
https://www.chrono24.at/search/index.htm?countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=18&maxAgeInDays=0&models=112&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
https://www.chrono24.at/search/index.htm?countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=18&maxAgeInDays=0&models=112&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
https://www.chrono24.at/search/index.htm?countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=18&maxAgeInDays=0&models=112&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
https://www.chrono24.at/search/index.htm?countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=18&maxAgeInDays=0&models=112&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
https://www.chrono24.at/search/index.htm?countryIds=AT&countryIds=CH&

In [110]:
alertnr = 3
sql_url_query = """SELECT watchurl, user_fk FROM alerter WHERE AlerterID = %s """
cur.execute(sql_url_query, (alertnr, ))
r = cur.fetchall()
for x in r:
    URL = str(x[0])
    user = x[1]
print(URL)
print(user)

https://www.chrono24.at/search/index.htm?caseMaterials=3&currencyId=EUR&dosearch=true&manufacturerIds=32&maxAgeInDays=0&models=24&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
2


In [ ]:
# End of for-loop:
conn.close()

https://www.chrono24.at/search/index.htm?caseMaterials=7&countryIds=AT&countryIds=CH&countryIds=DE&currencyId=EUR&dosearch=true&manufacturerIds=221&maxAgeInDays=0&models=2&pageSize=60&redirectToSearchIndex=true&resultview=block&sortorder=0
2
